In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
from ese.experiment.analysis.inference import load_cal_inference_stats

inference_paths = [
    root / "inference/01_09_24_ExpandedMetrics/WMH_Individual_Uncalibrated",
    root / "inference/01_09_24_ExpandedMetrics/WMH_Individual_TempScaling",
    root / "inference/01_09_24_ExpandedMetrics/WMH_Individual_VectorScaling",
    root / "inference/01_09_24_ExpandedMetrics/WMH_Individual_DirichletScaling",
    root / "inference/01_09_24_ExpandedMetrics/WMH_Individual_LTS",
    root / "inference/01_09_24_ExpandedMetrics/WMH_Ensemble_Uncalibrated",
    root / "inference/01_09_24_ExpandedMetrics/WMH_Ensemble_TempScaling",
    root / "inference/01_09_24_ExpandedMetrics/WMH_Ensemble_VectorScaling",
    root / "inference/01_09_24_ExpandedMetrics/WMH_Ensemble_DirichletScaling",
    root / "inference/01_09_24_ExpandedMetrics/WMH_Ensemble_LTS"
]

inference_info_dict = load_cal_inference_stats(
    log_dirs=inference_paths,
    load_image_df=True,
    load_pixel_meters_dict=False
)
image_info_df = inference_info_dict['image_info_df']
image_info_df = image_info_df.fillna('None')

In [ ]:
image_info_df['model._class'].unique()

In [ ]:
image_info_df[image_info_df['model._class'] == "ese.experiment.models.calibrators.Temperature_Scaling"]

In [ ]:
image_info_df['model._pretrained_class'].iloc[0]

In [ ]:
image_info_df.head(10)

In [ ]:
image_info_df["ensemble"] = image_info_df["model.ensemble"]
image_info_df["pre_softmax"] = image_info_df["model.ensemble_pre_softmax"]
image_info_df["combine_fn"] = image_info_df["model.ensemble_combine_fn"]

image_info_df["pretrained_seed"] = image_info_df["experiment.pretrained_seed"]

image_info_df["model_class"] = image_info_df["model._class"]
image_info_df["pretrained_model_class"] = image_info_df["model._pretrained_class"]

def method_name(model_class, pretrained_model_class, pretrained_seed, ensemble, pre_softmax, combine_fn):
    if ensemble:
        method_name_string = f"Ensemble (combined_fn={combine_fn}, pre_softmax={pre_softmax})" 
    else:
        if pretrained_model_class == "None":
            method_name_string = f"{model_class.split('.')[-1]} (seed={pretrained_seed})"
        else:
            method_name_string = f"{pretrained_model_class.split('.')[-1]} (seed={pretrained_seed})"

    return method_name_string

def model_class(model_class):
    if "UNet" in model_class:
        return "Uncalibrated"
    else:
        return model_class

image_info_df.augment(method_name)
image_info_df.augment(model_class)

In [ ]:
table_df = image_info_df.groupby(["method_name", "model_class", "qual_metric"])['qual_score'].mean().reset_index()
dice_table = table_df[table_df["qual_metric"] == "Dice"]
hd95_table = table_df[table_df["qual_metric"] == "HD95"]

In [ ]:
dice_table.pivot(index='method_name', columns='model_class', values='qual_score')